In [2]:
import pandas as pd
import pandasql as ps

In [20]:
df = pd.read_csv('nba_cleaned_data.csv')
#note NaN values for "quincy Douby in cleaned data set"
all_nba = pd.read_csv('data/End of Season Teams.csv')
awards = pd.read_csv('data/Player Award Shares.csv')


In [21]:
#remove early seasons
all_nba = ps.sqldf("""SELECT * FROM all_nba WHERE season >= 1980""")
awards = ps.sqldf("""SELECT * FROM awards WHERE season >= 1980 and winner = TRUE""")


In [22]:
query = """SELECT player_id, 
            COUNT(CASE WHEN type = 'All-NBA' then 1 END) AS 'ALL_NBA_total',
            COUNT(CASE WHEN type = 'All-Defense' then 1 END) AS 'ALL_DEF_total',
            COUNT(CASE WHEN type = 'All-NBA' AND number_tm = '1st' then 1 END) AS '1T_ALL_NBA_total',
            COUNT(CASE WHEN type = 'All-NBA' AND number_tm = '2nd' then 1 END) AS '2T_ALL_NBA_total',
            COUNT(CASE WHEN type = 'All-NBA' AND number_tm = '3rd' then 1 END) AS '3T_ALL_NBA_total',
            COUNT(CASE WHEN type = 'All-Defense' AND number_tm = '1st' then 1 END) AS '1T_ALL_DEF_total',
            COUNT(CASE WHEN type = 'All-Defense' AND number_tm = '2nd' then 1 END) AS '2T_ALL_DEF_total'
            FROM all_nba
            GROUP BY player_id"""

all_nba_totals = ps.sqldf(query)

In [23]:
query = """SELECT player_id, 
            COUNT(CASE WHEN award = 'dpoy' then 1 END) AS 'DPOY_total',
            COUNT(CASE WHEN award = 'mip' then 1 END) AS 'MIP_total',
            COUNT(CASE WHEN award = 'nba mvp' then 1 END) AS 'MVP_total',
            COUNT(CASE WHEN award = 'nba roy' then 1 END) AS 'ROY_total',
            COUNT(CASE WHEN award = 'smoy' then 1 END) AS 'SMOY_total'
            FROM awards
            GROUP BY player_id"""

award_totals = ps.sqldf(query)


In [24]:
query = """SELECT df.*, all_nba_totals.* FROM df LEFT JOIN all_nba_totals on df.player_id = all_nba_totals.player_id"""
df2 = ps.sqldf(query)
df2 = df2.drop('Unnamed: 0', axis =1)
df2 = df2.loc[:,~df2.columns.duplicated()].copy()


In [25]:
query = """SELECT df2.*, award_totals.* FROM df2 LEFT JOIN award_totals on df2.player_id = award_totals.player_id"""
df3 = ps.sqldf(query)
df3 = df3.loc[:,~df3.columns.duplicated()].copy()

In [26]:
df.columns[df.isnull().any()]

Index(['avg_ows_48', 'avg_dws_48', 'max_ows_48', 'max_dws_48'], dtype='object')

In [33]:
df3 = df3.fillna(0)

In [35]:
df3.to_csv('nba_cleaned_data_v2.csv')